In [48]:
import pandas as pd
import openpyxl
from pandas import DataFrame
from nltk.corpus import stopwords
from sklearn import tree
import pickle
import json
import re
import sklearn
import string

In [2]:
print('IMPORT INPUT CSV FILE IN CURRENT DIRECTORY AND NAME IT AS "IP_Data.csv"')

IMPORT INPUT CSV FILE IN CURRENT DIRECTORY AND NAME IT AS "IP_Data.csv"


In [3]:
input_df = pd.read_csv('IP_Data.csv')
length = input_df.shape[0]

In [4]:
f1 = open('dict_nlp_dom.pkl','rb')
dict_nlp_dom =pickle.load(f1)
f1.close()
f2 = open('dict_nlp_eve.pkl','rb')
dict_nlp_eve = pickle.load(f2)
f2.close()

In [5]:
name_list=[]
temp_name_list=[]
direct=[]
DE_direct=[]
EE_direct=[]

In [6]:
df = pd.read_csv('D:\\Cloud Counselage\\Data.csv')
copydf = df.copy()

In [7]:
dmUQ = df.Domain.unique()
dmNO=[]
[dmNO.append(x) for x in range(1,len(dmUQ)+1)]    

eeUQ = df.Event1.unique()
eeNO=[]
[eeNO.append(x) for x in range(1,len(eeUQ)+1)]
    
    
pUQ = df.Name.unique()
pNO=[]
[pNO.append(x) for x in range(1,len(pUQ)+1)]
               
df['Event1'].replace(eeUQ,eeNO,inplace=True)
df['Event2'].replace(eeUQ,eeNO,inplace=True)
df['Domain'].replace(dmUQ,dmNO,inplace=True)
df['Name'].replace(pUQ,pNO,inplace=True)
    
tmp_df=df.copy()

In [8]:
inst = open("Domain_extract_file.pkl","rb")
Domain_extract = pickle.load(inst)
inst2 = open("Event_extract_file.pkl","rb")
Event_extract = pickle.load(inst2)

In [9]:
event_list=[]
for x in range(0,length):
    event_list.append((input_df.iloc[x,0]).upper())
print(event_list)

['HANDS-ON MICROPROCESSOR HARDWARE WORKSHOP', 'AGILE DEVELOPMENT PROCESS BY PROF.XYZ', 'LENOVO SOFTWARE DEVELOPMENT WEBINAR BY XYZ TUTORIALS!', 'AI ML SEMINAR BY DEEPLEARNING ON 1ST AUGUST', 'MICROSOFT AZURE SEMINAR']


In [10]:
def dom_eve_extractor(t):
    flag_domain = 0
    words = t.split()
    
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    words = [re_punc.sub('-', x) for x in words]
    print(words)
    words = [re_punc.sub('', x) for x in words]
    print(words)

    DE= []
    DE_ml= []
    for x in words:
        try:
            DE.append(Domain_extract[x])
            flag_domain = 1
        except:
            pass
    if flag_domain == 0:
        for x in words:
            for y in words:
                try:
                    dummy = x+' '+y
                    DE.append(Domain_extract[dummy])
                except:
                    pass   
                
 
    nlp_df = DataFrame(list(dict_nlp_dom.items()),columns = ['Domain','txt']) 
    from sklearn.feature_extraction.text import CountVectorizer
    from nltk.corpus import stopwords
    vectorizer = CountVectorizer(stop_words='english',binary=True)
    train_vectors = vectorizer.fit_transform(nlp_df.txt)
    temp1 = vectorizer.transform([t])
    
    
    from sklearn import svm
    clf_svm = svm.SVC(kernel='rbf')
    clf_svm.fit(train_vectors,nlp_df.Domain)
    pred = clf_svm.predict(temp1)[0]
    DE_ml.append(pred)
    
    clf_svm = svm.SVC(kernel='linear')
    clf_svm.fit(train_vectors,nlp_df.Domain)
    pred = clf_svm.predict(temp1)[0]
    DE_ml.append(pred)
    
    from sklearn.tree import DecisionTreeClassifier
    clf_tree = DecisionTreeClassifier()
    clf_tree.fit(train_vectors,nlp_df.Domain)
    pred = clf_tree.predict(temp1)[0]
    DE_ml.append(pred)
    
    from sklearn.naive_bayes import GaussianNB
    clf_bay = GaussianNB()
    clf_bay.fit(train_vectors.toarray(),nlp_df.Domain)
    pred = clf_bay.predict(temp1.toarray())[0]
    DE_ml.append(pred)
    
    
    from sklearn.ensemble import RandomForestClassifier
    clf_RF = RandomForestClassifier()
    clf_RF.fit(train_vectors,nlp_df.Domain)
    pred = clf_RF.predict(temp1)[0]
    DE_ml.append(pred)
    
    
    for x in DE:
        if x not in DE_ml:
            DE_ml.append(x)
    for x in DE_ml:
        if x not in DE:
            if (len(DE_ml)>(len(DE)+1)):
                DE_ml.remove(x)
    DE_ml = list(dict.fromkeys(DE_ml))
    if (len(DE_ml) == 0):
        DE_ml.append('Other') 
     
    
    return DE_ml,DE


In [11]:
def eve_ext(t):
    words=t.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    words = [re_punc.sub('', x) for x in words]
    EE= []
    EE_ml=[]
    for x in words:
        try:            
            EE.append(Event_extract[x])
        except:
            pass
        
        
    nlp_df = DataFrame(list(dict_nlp_eve.items()),columns = ['Event','txt']) 
    from sklearn.feature_extraction.text import CountVectorizer
    from nltk.corpus import stopwords
    vectorizer = CountVectorizer(stop_words='english',binary=True)
    train_vectors = vectorizer.fit_transform(nlp_df.txt)
    temp1 = vectorizer.transform([t])
    
    from sklearn import svm
    clf_svm = svm.SVC(kernel='rbf')
    clf_svm.fit(train_vectors,nlp_df.Event)
    pred = clf_svm.predict(temp1)[0]
    EE_ml.append(pred)
    
    clf_svm = svm.SVC(kernel='linear')
    clf_svm.fit(train_vectors,nlp_df.Event)
    pred = clf_svm.predict(temp1)[0]
    EE_ml.append(pred)
    
    from sklearn.tree import DecisionTreeClassifier
    clf_tree = DecisionTreeClassifier()
    clf_tree.fit(train_vectors,nlp_df.Event)
    pred = clf_tree.predict(temp1)[0]
    EE_ml.append(pred)
    
    for x in EE:
        if x not in EE_ml:
            EE_ml.append(x)
        
    EE_ml = list(dict.fromkeys(EE_ml))
    return EE_ml,EE

## Training with Event 1

In [12]:
def domain_event(DE,EE):
    list_name=[]
    temp_list_names=[]
    for x in DE:
        for y in EE:
            tempcopy = copydf[copydf['Domain']==x].copy()
            tempcopy1 = tempcopy[tempcopy['Event1']==y].copy()
            tempcopy2 = tempcopy[tempcopy['Event2']==y].copy()
            tempcopy3 = pd.concat([tempcopy1,tempcopy2])

            
            for name in tempcopy3.Name:
                temp_list_names.append(name)
    return temp_list_names

In [13]:
words2num_flag = 0

for t in event_list:
    DE,DE_direct = dom_eve_extractor(t)
    EE,EE_direct = eve_ext(t)

    direct_tmp = domain_event(DE_direct,EE_direct)
    direct.append(direct_tmp)
    if len(t)==0:
        temp_name_list=['NONE']
    name_list.append(temp_name_list)
    temp_name_list=[]

['HANDS-ON', 'MICROPROCESSOR', 'HARDWARE', 'WORKSHOP']
['HANDSON', 'MICROPROCESSOR', 'HARDWARE', 'WORKSHOP']
['AGILE', 'DEVELOPMENT', 'PROCESS', 'BY', 'PROF-XYZ']
['AGILE', 'DEVELOPMENT', 'PROCESS', 'BY', 'PROFXYZ']
['LENOVO', 'SOFTWARE', 'DEVELOPMENT', 'WEBINAR', 'BY', 'XYZ', 'TUTORIALS-']
['LENOVO', 'SOFTWARE', 'DEVELOPMENT', 'WEBINAR', 'BY', 'XYZ', 'TUTORIALS']
['AI', 'ML', 'SEMINAR', 'BY', 'DEEPLEARNING', 'ON', '1ST', 'AUGUST']
['AI', 'ML', 'SEMINAR', 'BY', 'DEEPLEARNING', 'ON', '1ST', 'AUGUST']
['MICROSOFT', 'AZURE', 'SEMINAR']
['MICROSOFT', 'AZURE', 'SEMINAR']


In [30]:
print(name_list)

[[], [], [], [], []]


In [25]:
dict1 = {}
dict1_list=[]
for totalppl,event in zip(name_list,event_list):
    strxx = str(totalppl)[1:-1]
    dict1[event]=strxx

In [26]:
dfx = DataFrame(list(dict1.items()),columns = ['Events','Employee Name']) 
#dfx.drop(columns='index',inplace=True)
dfx

,Events,Employee Name
0,HANDS-ON MICROPROCESSOR HARDWARE WORKSHOP,"'Philip Woods', 'Russell Vargas', 'Edward Hamp..."
1,AGILE DEVELOPMENT PROCESS BY PROF.XYZ,"'Jennifer Thompson', 'Kristen Mitchell', 'Will..."
2,LENOVO SOFTWARE DEVELOPMENT WEBINAR BY XYZ TUT...,"'Jennifer Merritt', 'Robert Ramirez', 'David D..."
3,AI ML SEMINAR BY DEEPLEARNING ON 1ST AUGUST,"'Kristy Levine', 'Jasmine Brown', 'James Porte..."
4,MICROSOFT AZURE SEMINAR,"'Deborah Young', 'David Davis', 'Corey Beck', ..."


In [96]:
dfx.to_excel('CloudEvents.xlsx', index = None, header=True)

In [14]:
f = open('dict_nlp_dom.pkl','rb')
h1 = pickle.load(f)
f.close()

In [34]:
h1=

{'Management': 'Management managing manager manager administration government',
 'Coding': 'CODING code coder hackathon coding codeathon codes',
 'Security': 'cyber security hack ethical hacker',
 'Hardware': 'Hardware Processor computer storage memory battery graphics',
 'Web Development': 'Web development HTML CSS websites design layout stack',
 'Higher Education': 'education higher education graduate abroad study studies.',
 'IoT': 'IoT  Internet of things',
 'Artificial Intelligence': 'Artificial intelligence AI',
 'Finance': 'Finance banking investment economic invest save earn',
 'Mobile Applications': 'mobile application app android ios',
 'Software Architecture': 'software architecture structure',
 'Cloud Computing': 'Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure',
 'Networking': 'Networking  Network networks computer ports connect',
 'Machine Learning': 'Machine learning ml',
 'Blockchain': 'cryptocurrency Blockchain Block chain Bitcoin',
 'Development Processes': '

In [28]:
h2={'Management': 'Management managing manager manager administration government Management managing manager manager administration government Management managing manager manager administration government Management managing manager manager administration government Management managing manager manager administration government Management managing manager manager administration government Management managing manager manager administration government  Management managing manager manager administration government  Management managing manager manager administration government  Management managing manager manager administration government  Management managing manager manager administration government  Management managing manager manager administration government  Management managing manager manager administration government',
 'Coding': 'CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes CODING code coder hackathon coding codeathon codes',
 'Security': 'cyber security hack ethical hacker cyber security hack ethical hacker cyber security hack ethical hacker cyber security hack ethical hacker cyber security hack ethical hacker cyber security hack ethical hacker cyber security hack ethical hacker cyber security hack ethical hacker cyber security hack ethical hacker cyber security hack ethical hacker cyber security hack ethical hacker security hack ethical hacker cyber security hack ethical hacker security hack ethical hacker cyber security hack ethical hacker security hack ethical hacker cyber security hack ethical hacker security hack ethical hacker cyber security hack ethical hacker security hack ethical hacker cyber security hack ethical hacker security hack ethical hacker cyber security hack ethical hacker security hack ethical hacker cyber security hack ethical hacker security hack ethical hacker cyber security hack ethical hacker security hack ethical hacker cyber security hack ethical hacker',
 'Hardware': 'Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphicsHardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics Hardware Processor computer storage memory battery graphics',
 'Web Development': 'Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack Web development HTML CSS websites design layout stack',
 'Higher Education': 'education higher education graduate abroad study studies. education higher education graduate abroad study studies education higher education graduate abroad study studies education higher education graduate abroad study studies education higher education graduate abroad study studies education higher education graduate abroad study studies education higher education graduate abroad study studies education higher education graduate abroad study studies  abroad study studies education higher education graduate abroad study studies abroad study studies education higher education graduate abroad study studies abroad study studies education higher education graduate abroad study studies abroad study studies education higher education graduate abroad study studies abroad study studies education higher education graduate abroad study studies abroad study studies education higher education graduate abroad study studies abroad study studies education higher education graduate abroad study studies',
 'IoT': 'IoT  Internet of things IoT  Internet of things IoT  Internet of things IoT  Internet of things IoT  Internet of things IoT  Internet of things IoT  Internet of things IoT  Internet of things IoT  Internet of things IoT  Internet of things IoT  Internet of things Internet of things IoT  Internet of things IoT  Internet of things Internet of things IoT  Internet of things IoT  Internet of things Internet of things IoT  Internet of things IoT  Internet of things Internet of things IoT  Internet of things IoT  Internet of things Internet of things IoT  Internet of things IoT  Internet of things Internet of things IoT  Internet of things IoT  Internet of things Internet of things IoT  Internet of things IoT  Internet of things Internet of things IoT  Internet of things IoT  Internet of things Internet of things IoT  Internet of things IoT  Internet of things Internet of things IoT  Internet of things IoT  Internet of things Internet of things IoT  Internet of things IoT  Internet of things Internet of things IoT  Internet of things IoT  Internet of things',
 'Artificial Intelligence':  'AI  Artificial intelligence AI Artificial intelligence AI Artificial intelligence AI Artificial intelligence AI Artificial intelligence AI Artificial intelligence AI  Artificial intelligence AI Artificial intelligence  Artificial intelligence AI Artificial intelligence  Artificial intelligence AI Artificial intelligence  Artificial intelligence AI Artificial intelligence  Artificial intelligence AI Artificial intelligence  Artificial intelligence AI Artificial intelligence  Artificial intelligence AI Artificial intelligence  Artificial intelligence AI Artificial intelligence  Artificial intelligence AI Artificial intelligence  Artificial intelligence AI Artificial intelligence  Artificial intelligence AI Artificial intelligence Artificial intelligence AI Artificial intelligence  Artificial intelligence AI Artificial intelligence ',
 'Finance': 'Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earnFinance banking investment economic invest save earnFinance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn  Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn Finance banking investment economic invest save earn',
 'Mobile Applications': ' mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android iosmobile application app android ios mobile application app android iosmobile application app android iosmobile application app android iosmobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios  mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios mobile application app android ios',
 'Software Architecture': 'software architecture structure software architecture structure software architecture structure software architecture structure software architecture structuresoftware architecture structuresoftware architecture structuresoftware architecture structuresoftware architecture structuresoftware architecture structure software architecture structure software architecture structure software architecture structure software architecture structure  software architecture structure software architecture structure  software architecture structure software architecture structure  software architecture structure software architecture structure  software architecture structure software architecture structure  software architecture structure software architecture structure  software architecture structure software architecture structure  software architecture structure software architecture structure  software architecture structure software architecture structure  software architecture structure software architecture structure  software architecture structure software architecture structure  software architecture structure software architecture structure  software architecture structure software architecture structure  software architecture structure software architecture structure ',
 'Cloud Computing': ' Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft AzureCloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure  Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure Cloud computing AWS AZURE GOOGLE  Amazon Microsoft Azure',
 'Networking': 'Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connectNetworking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect Networking  Network networks computer ports connect',
 'Machine Learning': 'ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml  Machine learning ml Machine learning ml Machine learning ml Machine learning ml  Machine learning ml Machine learning ml  Machine learning ml Machine learning ml  Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml Machine learning ml  Machine learning ml Machine learning ml Machine learning ml Machine learning ml  Machine learning ml Machine learning ml  Machine learning ml Machine learning ml ',
 'Blockchain': 'cryptocurrency Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin  Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin Blockchain Block chain Bitcoin cryptocurrency Blockchain Block chain Bitcoin',
 'Development Processes': ' Development software processes process SDLC agile  Development software processes process SDLC agile  Development software processes process SDLC agile  Development software processes process SDLC agile  Development software processes process SDLC agile  Development software processes process SDLC agile  Development software processes process SDLC agile  Development software processes process SDLC agile   Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile    Development software processes process SDLC agile  ',
 'Data Science': ' data scientists science big IBM kaggle. data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle  data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientists science big IBM kaggle data scientist science big IBM kaggle data scientist science big IBM kaggle data scientist science big IBM kaggle data scientists science big IBM kaggle data scientist science big IBM kaggle data scientists science big IBM kaggle'}

In [55]:
listdict=[]
listdictno=[]
i=0
for x in h1:
    listdict.append(x)
    listdictno.append(i)
    i+=1

In [56]:
listdict

['Management',
 'Coding',
 'Security',
 'Hardware',
 'Web Development',
 'Higher Education',
 'IoT',
 'Artificial Intelligence',
 'Finance',
 'Mobile Applications',
 'Software Architecture',
 'Cloud Computing',
 'Networking',
 'Machine Learning',
 'Blockchain',
 'Development Processes',
 'Data Science']

In [73]:
nlp_df = DataFrame(list(h1.items()),columns = ['Domain','txt']) 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
vectorizer = CountVectorizer(stop_words='english',binary=True)
train_vectors = vectorizer.fit_transform(nlp_df.txt)
temp1 = vectorizer.transform(['finance'])
print(temp1)

  (0, 38)	1


In [74]:
from sklearn.tree import DecisionTreeClassifier
clf_tree = DecisionTreeClassifier()
clf_tree.fit(train_vectors,nlp_df.Domain)
pred = clf_tree.predict_proba(temp1)[0]
print(pred)
for index, prob in enumerate(pred):
    if prob>=0.3:
        for x,y in zip(pNO,listdict):
            if x==index+1:
                print(y)
    
from sklearn.naive_bayes import GaussianNB
clf_bay = GaussianNB()
clf_bay.fit(train_vectors.toarray(),nlp_df.Domain)
pred = clf_bay.predict(temp1.toarray())[0]
print(pred)
    

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Blockchain
Artificial Intelligence


In [75]:
from sklearn.naive_bayes import GaussianNB
clf_bay = GaussianNB()
clf_bay.fit(train_vectors.toarray(),nlp_df.Domain)
pred = clf_bay.predict(temp1.toarray())[0]
print(pred)

Artificial Intelligence
